In [1]:
import json
import pandas as pd
train = pd.read_json('../input/whats-cooking-kernels-only/train.json')
test = pd.read_json('../input/whats-cooking-kernels-only/test.json')

In [2]:
from sklearn.model_selection import train_test_split
X_train = list(train['ingredients'])
y_train = list(train['cuisine'])
X_test = list(test['ingredients'])
X_C_train, X_C_val, y_C_train, y_C_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [3]:
print(len(y_train))
print(len(set(y_train)))

39774
20


In [4]:
from nltk.corpus import stopwords 
stop_words = set(stopwords.words('english'))

In [5]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

In [6]:
from nltk.tokenize import word_tokenize
def preprocess_ingredient(ingredients):
    for ingredient in ingredients:
        for i in range(len(ingredient)):
            x = ingredient[i] #'Bertolli® Classico Olive Oil', '(10 oz.) frozen chopped spinach, thawed and squeezed dry' ,'leg of lamb', 'lamb leg'
            x = x.lower() #'bertolli® classico olive oil', '(10 oz.) frozen chopped spinach, thawed and squeezed dry' ,'leg of lamb', 'lamb leg'
            x = re.sub("[^a-z ]", "", x) #'bertolli classico olive oil', ' oz frozen chopped spinach thawed and squeezed dry' ,'leg of lamb', 'lamb leg'
            word_tokens = word_tokenize(x)
            if 'oz' in word_tokens:
                word_tokens.remove('oz')
            filtered_words = [w for w in word_tokens if not w in stop_words] 
            filtered_words.sort() #['bertolli', 'classico', 'oil', 'olive'], ['chopped', 'dry', 'frozen', 'spinach', 'squeezed', 'thawed'], ['lamb', 'leg'], ['lamb', 'leg']
            stemmed_word = [stemmer.stem(word) for word in filtered_words]
            x = ' '.join(stemmed_word) #'bertolli classico oil oliv', 'chop dri frozen spinach squeez thaw' ,'lamb leg', 'lamb leg'
            ingredient[i] = x

In [7]:
preprocess_ingredient(X_C_train)
preprocess_ingredient(X_C_val)
preprocess_ingredient(X_train)
preprocess_ingredient(X_test)

In [8]:
def create_vocabs(X):
    vocabs = set()
    for ingredient in X:
        vocabs.update(ingredient)
    return sorted(vocabs)

In [9]:
C_train_vocabs = create_vocabs(X_C_train)
train_vocabs = create_vocabs(X_train)

In [10]:
import pickle
with open('train_vocabs.pkl', 'wb') as file:
    pickle.dump(train_vocabs, file)
# with open('train_vocabs.pkl', 'rb') as file:  
#     pickled_train_vocabs = sorted(pickle.load(file))

In [11]:
def create_bag_of_words(ingredients, vocabs):
    data_features = list()
    for ingredient in ingredients:
        features = list()
        for item in vocabs:
            features.append(item in ingredient)
        data_features.append(features)
    return data_features

In [12]:
# from sklearn import preprocessing

C_train_data_features = create_bag_of_words(X_C_train, C_train_vocabs)

C_val_data_features = create_bag_of_words(X_C_val, C_train_vocabs)

# train_data_features = preprocessing.scale(create_bag_of_words(X_train))
train_data_features = create_bag_of_words(X_train, train_vocabs)

# test_data_features = preprocessing.scale(create_bag_of_words(X_test))
test_data_features = create_bag_of_words(X_test, train_vocabs)

In [13]:
def train_logistic_regression(c, features, label):
    from sklearn.linear_model import LogisticRegression
    ml_model = LogisticRegression(C=c, max_iter = 40000)
    ml_model.fit(features, label)
    return ml_model

In [14]:
c_trial = [0.01, 0.1, 0.2, 0.5, 1, 2, 5, 10, 100]
best_c = c = 0
best_score = 0
from sklearn.metrics import accuracy_score
for c in c_trial:
    ml_model = train_logistic_regression(c, C_train_data_features, y_C_train)
    predicted_y = ml_model.predict(C_val_data_features)
    score = accuracy_score(y_C_val, predicted_y)
    print(c, score)
    if score > best_score:
        best_score = score
        best_c = c

print(best_c, best_score)
ml_model = train_logistic_regression(best_c, train_data_features, y_train)
predicted_y = ml_model.predict(test_data_features)

0.01 0.6517913262099309
0.1 0.7581395348837209
0.2 0.7730986800754243
0.5 0.780389692017599
1 0.7827781269641735
2 0.77950974230044
5 0.7717159019484601
10 0.7629164047768698
100 0.7263356379635449
1 0.7827781269641735


In [15]:
test['cuisine'] = predicted_y
test[['id', 'cuisine']].to_csv('submission.csv', index=False)
test[['id', 'cuisine']].head()

,id,cuisine
0,18009,british
1,28583,southern_us
2,41580,italian
3,29752,cajun_creole
4,35687,italian


In [16]:
# import pickle
# with open('cuisine_prediction_model.pkl', 'wb') as file:
#     pickle.dump(ml_model, file)
    
# with open('cuisine_prediction_model.pkl', 'rb') as file:  
#     pickled_ml_model = pickle.load(file)

import joblib
joblib.dump(ml_model, 'cuisine_prediction_model_joblib.pkl')
# pickled_ml_model = joblib.load('cuisine_prediction_model_joblib.pkl') 

['cuisine_prediction_model_joblib.pkl']

In [17]:
# !pip freeze > '../working/dockerimage_snapshot.txt'